In [96]:
from keras.applications import VGG16 , InceptionV3
from keras.applications.resnet import ResNet101
import keras
from keras.models import Model
from keras.layers import Dense,Flatten,Dropout
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/Train",target_size=(224,224))
valdata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/Test", target_size=(224,224))
testdata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/TestNew", target_size=(224,224))
    



model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in model.layers:
    layer.trainable = False

    

    
x = model.output
x = Flatten()(x) 
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x) 

x = Dense(256, activation='relu')(x)
x = Dense(3, activation='softmax')(x) 

transfer_model = Model(inputs= model.input, outputs=x)

transfer_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001 , momentum=0.9), metrics=["accuracy"])

transfer_model.summary()

from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("resnet.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=40, verbose=1, mode='auto')
hist = transfer_model.fit_generator(generator= traindata, epochs= 10, validation_data= testdata,  callbacks=[checkpoint,early])
transfer_model.save_weights("ResNet.h5")


Found 9384 images belonging to 3 classes.
Found 3128 images belonging to 3 classes.
Found 3615 images belonging to 3 classes.
Model: "model_41"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_44 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_44[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                               

In [97]:
from keras.applications import VGG16 , InceptionV3
from keras.applications.resnet import ResNet101
import keras
from keras.models import Model
from keras.layers import Dense,Flatten,Dropout
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/Train",target_size=(224,224))
valdata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/Test", target_size=(224,224))
testdata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/TestNew", target_size=(224,224))
    



model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in model.layers:
    layer.trainable = False

    

    
x = model.output
x = Flatten()(x) 
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x) 

x = Dense(256, activation='relu')(x)
x = Dense(3, activation='softmax')(x) 

transfer_model = Model(inputs= model.input, outputs=x)

transfer_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001 , momentum=0.9), metrics=["accuracy"])

transfer_model.summary()

# from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint = ModelCheckpoint("mobilenet.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=40, verbose=1, mode='auto')
# hist = transfer_model.fit_generator(generator= traindata, epochs= 10, validation_data= testdata,  callbacks=[checkpoint,early])
# transfer_model.save_weights("mobilenet.h5")


Found 9384 images belonging to 3 classes.
Found 3128 images belonging to 3 classes.
Found 3615 images belonging to 3 classes.
Model: "model_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_45 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                